In [2]:
# Change directory to the desired path
%cd D:\Documents\ai_engineer_pacmann\ai_engineer_pacmann\CANDRA_MLPROCESS\src

import utils

from utils import load_data
from utils import split_input_output
from utils import split_train_test
from utils import serialize_data
from utils import deserialize_data

from copy import deepcopy
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd


c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


D:\Documents\ai_engineer_pacmann\ai_engineer_pacmann\CANDRA_MLPROCESS\src


In [3]:
# Alamat file X dan y untuk di deserialize

X_TRAIN_PATH = '..\data\interim\X_train.pkl'
Y_TRAIN_PATH = '..\data\interim\y_train.pkl'
X_TEST_PATH = '..\data\interim\X_test.pkl'
Y_TEST_PATH = '..\data\interim\y_test.pkl'
X_VAL_PATH = '..\data\interim\X_valid.pkl'
Y_VAL_PATH = '..\data\interim\y_valid.pkl'

In [24]:
# Melakukan deserialize data agar dapat digunakan didalam variable

X_train = deserialize_data(path=X_TRAIN_PATH)
y_train = deserialize_data(path=Y_TRAIN_PATH)
X_test = deserialize_data(path=X_TEST_PATH)
y_test = deserialize_data(path=Y_TEST_PATH)
X_valid = deserialize_data(path=X_VAL_PATH)
y_valid = deserialize_data(path=Y_VAL_PATH)

In [25]:
X_train.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
15884,25,241875,MORTGAGE,4.0,EDUCATION,A,16000,7.05,0.07,N,4
15138,21,18000,RENT,5.0,PERSONAL,B,1500,12.18,0.08,N,4
7474,25,53000,MORTGAGE,10.0,MEDICAL,B,16000,12.53,0.30,N,2
18212,28,16800,OWN,NaN,MEDICAL,C,5000,13.98,0.30,N,8
6493,25,50000,MORTGAGE,2.0,VENTURE,A,10000,7.90,0.20,N,2


In [6]:
y_train.head()

15884    0
15138    1
7474     0
18212    1
6493     0
Name: loan_status, dtype: int64

In [7]:
X_test.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
20439,30,31000,RENT,7.0,MEDICAL,D,4750,14.84,0.15,N,9
30809,36,60000,RENT,3.0,EDUCATION,A,9500,7.14,0.16,N,12
22978,29,33000,RENT,5.0,HOMEIMPROVEMENT,A,8000,5.79,0.24,N,8
19435,27,40000,RENT,3.0,PERSONAL,A,3000,7.05,0.07,N,8
31011,39,56000,RENT,16.0,DEBTCONSOLIDATION,C,10000,13.06,0.18,N,11


# d. Buang Data duplicate X_train y_train


In [26]:
def drop_duplicate_data(X, y):
    """
    Fungsi untuk membuang data duplikat dari dataset.
    
    Parameters:
    X : pd.DataFrame
        DataFrame yang berisi data fitur.
    y : pd.Series
        Series yang berisi data target yang sesuai dengan data fitur pada X.
    
    Returns:
    X : pd.DataFrame
        DataFrame yang telah dibuang data duplikatnya.
    y : pd.Series
        Series yang telah disesuaikan dengan data fitur setelah membuang data duplikat.
    """
    
    # Validasi parameter X
    if not isinstance(X, pd.DataFrame):
        raise ValueError("Parameter X harus bertipe DataFrame.")
    
    # Validasi parameter y
    if not isinstance(y, pd.Series):
        raise ValueError("Parameter y harus bertipe Series.")
    
    print("Fungsi drop_duplicate_data: parameter telah divalidasi.")
    
    # Copy data
    X = X.copy()
    y = y.copy()
    
    # Print shape sebelum dropping duplicate
    print(f"Fungsi drop_duplicate_data: shape dataset sebelum dropping duplicate adalah {X.shape}.")
    
    # Pengecekan duplicate
    X_duplicate = X[X.duplicated(keep=False)]
    print(f"Fungsi drop_duplicate_data: shape dari data yang duplicate adalah {X_duplicate.shape}.")
    
    # Kalkulasi shape setelah drop duplicate
    X_clean = (X.shape[0] - X_duplicate.shape[0], X.shape[1])
    print(f"Fungsi drop_duplicate_data: shape dataset setelah drop duplicate seharusnya adalah {X_clean}.")
    
    # Drop duplicate
    X.drop_duplicates(inplace=True)
    
    # Seleksi data y dengan index dari data X yang telah didrop duplicate
    y = y[X.index]
    
    # Print shape setelah dropping duplicate
    print(f"Fungsi drop_duplicate_data: shape dataset setelah dropping duplicate adalah {X.shape}.")
    
    return X, y

In [27]:
# Memanggil fungsi untuk membuang data duplikat pada dataset X_train y_train
X_train, y_train = drop_duplicate_data(X=X_train, y=y_train)


Fungsi drop_duplicate_data: parameter telah divalidasi.
Fungsi drop_duplicate_data: shape dataset sebelum dropping duplicate adalah (26064, 11).
Fungsi drop_duplicate_data: shape dari data yang duplicate adalah (192, 11).
Fungsi drop_duplicate_data: shape dataset setelah drop duplicate seharusnya adalah (25872, 11).
Fungsi drop_duplicate_data: shape dataset setelah dropping duplicate adalah (25968, 11).


# e. Imputasi Data Numerik

In [28]:
#import pandas as pd
#from copy import deepcopy

def median_imputation(data, subset_data, fit):
    """
    Fungsi untuk melakukan imputasi data numerik dengan nilai median.
    
    Parameters:
    data : pd.DataFrame
        DataFrame yang berisi data yang ingin diimputasi.
    subset_data : list atau dict
        Jika fit = True, subset_data adalah daftar nama kolom yang ingin dicari nilai mediannya.
        Jika fit = False, subset_data adalah dict dengan nama kolom sebagai key dan nilai median sebagai value.
    fit : bool
        Jika True, melakukan kalkulasi nilai median dari tiap kolom yang dispesifikkan pada subset_data.
        Jika False, melakukan imputasi pada data berdasarkan kalkulasi sebelumnya.
        
    Returns:
    Jika fit = True, mengembalikan dict dengan nama kolom dan nilai median.
    Jika fit = False, mengembalikan DataFrame dengan data yang telah diimputasi.
    """
    
    # Validasi parameter data
    if not isinstance(data, pd.DataFrame):
        raise RuntimeError("Fungsi median_imputation: parameter data haruslah bertipe DataFrame!")
    
    # Validasi parameter fit dan subset_data
    if fit == True:
        if not isinstance(subset_data, list):
            raise RuntimeError("Fungsi median_imputation: untuk nilai parameter fit = True, subset_data harus bertipe list dan berisi daftar nama kolom yang ingin dicari nilai mediannya guna menjadi data imputasi pada kolom tersebut.")
    elif fit == False:
        if not isinstance(subset_data, dict):
            raise RuntimeError("Fungsi median_imputation: untuk nilai parameter fit = False, subset_data harus bertipe dict dan berisi key yang merupakan nama kolom beserta value yang merupakan nilai median dari kolom tersebut.")
    else:
        raise RuntimeError("Fungsi median_imputation: parameter fit haruslah bertipe boolean, bernilai True atau False.")
    
    print("Fungsi median_imputation: parameter telah divalidasi.")
    
    # Copy data
    data = data.copy()
    subset_data = deepcopy(subset_data)
    
    # Imputasi data
    if fit == True:
        imputation_data = {}
        for subset in subset_data:
            median_value = data[subset].median()
            imputation_data[subset] = median_value
        print(f"Fungsi median_imputation: proses fitting telah selesai, berikut hasilnya {imputation_data}.")
        return imputation_data
    else:
        print("Fungsi median_imputation: informasi count na sebelum dilakukan imputasi:")
        print(data.isna().sum())
        print("")
        data.fillna(subset_data, inplace=True)
        print("Fungsi median_imputation: informasi count na setelah dilakukan imputasi:")
        print(data.isna().sum())
        print("")
        return data

In [29]:
# Menentukan subset_data
subset_data = ['person_age' , 'person_income', 'person_emp_length', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length']

# Panggil fungsi median_imputation() untuk fitting
subset_data = median_imputation(X_train, subset_data, fit=True)

Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: proses fitting telah selesai, berikut hasilnya {'person_age': np.float64(26.0), 'person_income': np.float64(55000.0), 'person_emp_length': np.float64(4.0), 'loan_amnt': np.float64(8000.0), 'loan_int_rate': np.float64(10.99), 'loan_percent_income': np.float64(0.15), 'cb_person_cred_hist_length': np.float64(4.0)}.


In [30]:
# Panggil fungsi median_imputation() untuk imputasi pada X_train
X_train = median_imputation(X_train, subset_data, fit=False)

Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: informasi count na sebelum dilakukan imputasi:
person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              734
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 2491
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64

Fungsi median_imputation: informasi count na setelah dilakukan imputasi:
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64



In [31]:
# Panggil fungsi median_imputation() untuk imputasi pada X_test
X_test = median_imputation(X_test, subset_data, fit=False)

Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: informasi count na sebelum dilakukan imputasi:
person_age                      0
person_income                   0
person_home_ownership           0
person_emp_length              77
loan_intent                     0
loan_grade                      0
loan_amnt                       0
loan_int_rate                 303
loan_percent_income             0
cb_person_default_on_file       0
cb_person_cred_hist_length      0
dtype: int64

Fungsi median_imputation: informasi count na setelah dilakukan imputasi:
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64



In [32]:
# Panggil fungsi median_imputation() untuk imputasi pada X_valid
X_valid = median_imputation(X_valid, subset_data, fit=False)

Fungsi median_imputation: parameter telah divalidasi.
Fungsi median_imputation: informasi count na sebelum dilakukan imputasi:
person_age                      0
person_income                   0
person_home_ownership           0
person_emp_length              80
loan_intent                     0
loan_grade                      0
loan_amnt                       0
loan_int_rate                 312
loan_percent_income             0
cb_person_default_on_file       0
cb_person_cred_hist_length      0
dtype: int64

Fungsi median_imputation: informasi count na setelah dilakukan imputasi:
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64



# f. Encoding data kategorikal - Fit OHE

In [40]:
# Fungsi untuk membuat OneHotEncoder dan menyimpannya ke directory computer
def create_onehot_encoder(categories, path):
    """
    Fungsi untuk membuat OneHotEncoder dan menyimpannya ke directory.
    
    Parameters:
    categories : list
        Daftar kategorik yang akan dibuat encodernya.
    path : str
        Lokasi pada disk komputer dimana encoder yang dibuat akan disimpan.
        
    Returns:
    ohe : OneHotEncoder
        Instance dari OneHotEncoder yang telah difit dengan data kategori.
    """
    
    # Validasi parameter categories
    if not isinstance(categories, list):
        raise RuntimeError("Fungsi create_onehot_encoder: parameter categories haruslah bertipe list, berisi kategori yang akan dibuat encodernya.")
    
    # Validasi parameter path
    if not isinstance(path, str):
        raise RuntimeError("Fungsi create_onehot_encoder: parameter path haruslah bertipe string, berisi lokasi pada disk komputer dimana encoder akan disimpan.")
    
    # Buat instance OneHotEncoder
    ohe = OneHotEncoder(handle_unknown='ignore')
    
    # Melakukan fitting encoder
    categories_array = np.array(categories).reshape(-1, 1)
    ohe.fit(categories_array)
    
    # Menyimpan encoder ke disk
    serialize_data(ohe, path)
    
    # Print kategori yang telah dipelajari
    learned_categories = ohe.categories_[0].tolist()
    print(f"Kategori yang telah dipelajari adalah {learned_categories}")
    
    return ohe

In [38]:
print(X_train['person_home_ownership'].unique())
print(X_train['loan_intent'].unique())
print(X_train['loan_grade'].unique())
print(X_train['cb_person_default_on_file'].unique())


['MORTGAGE' 'RENT' 'OWN' 'OTHER']
['EDUCATION' 'PERSONAL' 'MEDICAL' 'VENTURE' 'HOMEIMPROVEMENT'
 'DEBTCONSOLIDATION']
['A' 'B' 'C' 'D' 'F' 'E' 'G']
['N' 'Y']


In [43]:
# Daftar kategorik dari kolom masing-masing
person_home_ownership = ['MORTGAGE' 'RENT' 'OWN' 'OTHER']
loan_intent = ['EDUCATION','PERSONAL','MEDICAL','VENTURE','HOMEIMPROVEMENT','DEBTCONSOLIDATION']
loan_grade = ['A', 'B', 'C', 'D', 'F', 'E', 'G']
cb_person_default_on_file = ['N', 'Y']


# Membuat encoder untuk masing-masing kategorik dan menyimpannya
ohe_home_ownership = create_onehot_encoder(person_home_ownership, '../models/ohe_home_ownership.pkl')
ohe_loan_intent = create_onehot_encoder(loan_intent, '../models/ohe_loan_intent.pkl')
ohe_loan_grade = create_onehot_encoder(loan_grade, '../models/ohe_loan_grade.pkl')
ohe_default_on_file = create_onehot_encoder(cb_person_default_on_file, '../models/ohe_default_on_file.pkl')

Kategori yang telah dipelajari adalah ['MORTGAGERENTOWNOTHER']
Kategori yang telah dipelajari adalah ['DEBTCONSOLIDATION', 'EDUCATION', 'HOMEIMPROVEMENT', 'MEDICAL', 'PERSONAL', 'VENTURE']
Kategori yang telah dipelajari adalah ['A', 'B', 'C', 'D', 'E', 'F', 'G']
Kategori yang telah dipelajari adalah ['N', 'Y']


In [42]:
%pwd

'D:\\Documents\\ai_engineer_pacmann\\ai_engineer_pacmann\\CANDRA_MLPROCESS\\src'

# Transform OHE

In [44]:
def ohe_transform(dataset, subset, prefix, ohe):
    """
    Function to apply OneHotEncoder to a specified column in a dataset.
    
    Parameters:
    dataset : pd.DataFrame
        The dataset to which encoding will be applied.
    subset : str
        The name of the column in the dataset to be encoded.
    prefix : str
        The prefix to be added to the names of the new encoded columns.
    ohe : OneHotEncoder
        The OneHotEncoder instance that has been fitted to the categorical data.
        
    Returns:
    dataset : pd.DataFrame
        The dataset with the encoded column.
    """
    
    # Validate parameters
    if not isinstance(dataset, pd.DataFrame):
        raise RuntimeError("Fungsi ohe_transform: parameter dataset harus bertipe DataFrame!")
    
    if not isinstance(ohe, OneHotEncoder):
        raise RuntimeError("Fungsi ohe_transform: parameter ohe harus bertipe OneHotEncoder!")
    
    if not isinstance(prefix, str):
        raise RuntimeError("Fungsi ohe_transform: parameter prefix harus bertipe str!")
    
    if not isinstance(subset, str):
        raise RuntimeError("Fungsi ohe_transform: parameter subset harus bertipe str!")
    
    try:
        dataset.columns.tolist().index(subset)
    except ValueError:
        raise RuntimeError("Fungsi ohe_transform: parameter subset string namun data tidak ditemukan dalam daftar kolom yang terdapat pada parameter dataset.")
    
    print("Fungsi ohe_transform: parameter telah divalidasi.")
    
    # Duplicate the dataset to avoid modifying the original data
    dataset = dataset.copy()
    
    print(f"Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah {list(dataset.columns)}.")
    
    # Create new column names for the encoded columns
    col_names = [f"{prefix}_{col_name}" for col_name in ohe.categories_[0].tolist()]
    
    # Perform encoding
    encoded = pd.DataFrame(ohe.transform(dataset[[subset]]).toarray(), columns=col_names, index=dataset.index)
    
    # Concatenate the original dataset with the new encoded columns
    dataset = pd.concat([dataset, encoded], axis=1)
    
    # Drop the original column that was encoded
    dataset.drop(columns=[subset], inplace=True)
    
    print(f"Fungsi ohe_transform: daftar nama kolom setelah dilakukan pengkodean adalah {list(dataset.columns)}.")
    
    return dataset


In [45]:
# Encoding untuk X_train
X_train = ohe_transform(X_train, "person_home_ownership", "home_ownership", ohe_home_ownership)
X_train = ohe_transform(X_train, "loan_intent", "loan_intent", ohe_loan_intent)
X_train = ohe_transform(X_train, "loan_grade", "loan_grade", ohe_loan_grade)
X_train = ohe_transform(X_train, "cb_person_default_on_file", "default_onfile", ohe_default_on_file)

Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_home_ownership', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length'].
Fungsi ohe_transform: daftar nama kolom setelah dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'home_ownership_MORTGAGERENTOWNOTHER'].
Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'home_ownership_MORTGAGE

c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [46]:
# Encoding untuk X_test
X_test = ohe_transform(X_test, "person_home_ownership", "home_ownership", ohe_home_ownership)
X_test = ohe_transform(X_test, "loan_intent", "loan_intent", ohe_loan_intent)
X_test = ohe_transform(X_test, "loan_grade", "loan_grade", ohe_loan_grade)
X_test = ohe_transform(X_test, "cb_person_default_on_file", "default_onfile", ohe_default_on_file)

Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_home_ownership', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length'].
Fungsi ohe_transform: daftar nama kolom setelah dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'home_ownership_MORTGAGERENTOWNOTHER'].
Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'home_ownership_MORTGAGE

c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [47]:
# Encoding untuk X_valid
X_valid = ohe_transform(X_valid, "person_home_ownership", "home_ownership", ohe_home_ownership)
X_valid = ohe_transform(X_valid, "loan_intent", "loan_intent", ohe_loan_intent)
X_valid = ohe_transform(X_valid, "loan_grade", "loan_grade", ohe_loan_grade)
X_valid = ohe_transform(X_valid, "cb_person_default_on_file", "default_onfile", ohe_default_on_file)

Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_home_ownership', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length'].
Fungsi ohe_transform: daftar nama kolom setelah dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'home_ownership_MORTGAGERENTOWNOTHER'].
Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'home_ownership_MORTGAGE

c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\lolcandra\miniconda3\envs\mlproces-pacmann\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


# g. Serialize Data


In [49]:
# Call the serialize function to dump the dataset into .pkl files

serialize_X_train = serialize_data(data=X_train,
                                  path='../data/processed/X_train_prep.pkl')

serialize_x_test = serialize_data(data=X_test,
                               path='../data/processed/X_test_prep.pkl')

serialize_x_valid = serialize_data(data=X_valid,
                               path='../data/processed/X_valid_prep.pkl')


In [50]:
# Call the serialize function to dump the dataset into .pkl files

serialize_y_train = serialize_data(data=y_train,
                                  path='../data/processed/y_train_prep.pkl')

In [51]:
print(f"X_train_prep shape: {X_train.shape}")
print(f"y_train_prep shape: {y_train.shape}")

X_train_prep shape: (25968, 23)
y_train_prep shape: (25968,)
